In [ ]:
!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

In [ ]:
!pip install ipyplot -q

In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
import glob
import os

import os
import numpy as np
import pandas as pd

import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

from sklearn.preprocessing import MultiLabelBinarizer

from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from fastai.vision.all import *
import ipyplot
import imageio
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))

colours = ['_red.png', '_blue.png', '_yellow.png', '_green.png']
TRAIN = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(TRAIN, train.iloc[idx,0])+ colour for colour in colours] for idx in range(len(train))]

In [ ]:
LABELS= {
    0: "Nucleoplasm",
    1: "Nuclear membrane",
    2: "Nucleoli",
    3: "Nucleoli fibrillar center",
    4: "Nuclear speckles",
    5: "Nuclear bodies",
    6: "Endoplasmic reticulum",
    7: "Golgi apparatus",
    8: "Intermediate filaments",
    9: "Actin filaments",
    10: "Microtubules",
    11: "Mitotic spindle",
    12: "Centrosome",
    13: "Plasma membrane",
    14: "Mitochondria",
    15: "Aggresome",
    16: "Cytosol",
    17: "Vesicles and punctate cytosolic patterns",
    18: "Negative"
}

In [ ]:
train_csv = train.copy()
train_csv['Label'] = train_csv['Label'].apply(lambda x: list(map(int,x.split("|"))))
mlb = MultiLabelBinarizer()
train_csv[list(range(19))] = mlb.fit_transform(train_csv['Label'])
train_csv.columns = ["ID", "Label"] + list(LABELS.values())

In [ ]:
train

In [ ]:
train_csv = train.copy()
train_csv['Label'] = train_csv['Label'].apply(lambda x: list(map(int,x.split("|"))))
mlb = MultiLabelBinarizer()
train_csv[list(range(19))] = mlb.fit_transform(train_csv['Label'])
train_csv.columns = ["ID", "Label"] + list(LABELS.values())
train_csv.head(5)

In [ ]:
label_count = train_csv.iloc[:, 2:].sum()
px.bar(label_count)

In [ ]:
train['num_classes'] = train['Label'].apply(lambda r: len(r.split('|')))
train['num_classes'].value_counts().plot.bar(title='Examples with multiple labels', xlabel='number of labels per example', ylabel='# train examples')
plt.show()

In [ ]:

labels = [str(i) for i in range(19)]
class_images = []
for label in labels:
    r_img = train[train.Label == label].reset_index(drop=True).ID.loc[0] + '_red.png'
    y_img = train[train.Label == label].reset_index(drop=True).ID.loc[0] + '_yellow.png'
    b_img = train[train.Label == label].reset_index(drop=True).ID.loc[0] + '_blue.png'
    r = imageio.imread(TRAIN + '/' + r_img)
    y = imageio.imread(TRAIN + '/' + y_img)
    b = imageio.imread(TRAIN + '/' + b_img)
    rgb = np.dstack((r,y,b))
    class_images.append(PILImage.create(rgb))

ipyplot.plot_images(images=class_images, labels=list(LABELS.values()), max_images=19, img_width=300)

In [ ]:


for i in range(3):
    img_visual = TRAIN + '/' + train_csv['ID'].iloc[i]
    r = plt.imread(img_visual + '_red' + '.png')
    g = plt.imread(img_visual + '_green' + '.png')    
    b = plt.imread(img_visual + '_blue' + '.png')
    y = plt.imread(img_visual + '_yellow' + '.png')
    fig, ax = plt.subplots(1,4, figsize=(10,20))
    img = np.dstack((r, g, b, y))
    ax[0].set_title('r_g_b_y')
    ax[0].imshow(img)
    ax[0].axis('off')

    img = np.dstack((r, g, b))
    ax[1].set_title('r_g_b')
    ax[1].imshow(img)
    ax[1].axis('off')

    img = np.dstack((r, y, b))
    ax[2].set_title('r_y_b')
    ax[2].imshow(img)
    ax[2].axis('off')

    img = np.dstack((b, y, g))
    ax[3].set_title("b_y_g")
    ax[3].imshow(img)
    ax[3].axis('off')

    plt.show()


In [ ]:
# Input: list of image filters as png
# Output: list of image filters as np.arrays
def image_to_arrays(path):
    
    image_arrays = list()
    for image in path:
        array = np.asarray(Image.open(image))
        image_arrays.append(array)
        
    return image_arrays

In [ ]:
# Introduce list of image filters
# Returns a processed image ready for the CNN and an encoded label as tensor
def image_prep(paths, label):

    img = image_to_arrays(paths)
    size = np.shape(img[0])[0]
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    img = tf.reshape(img, (1, size, size, 3))
    img = tf.image.resize(img, IMG_SIZE)

    label = tf.strings.split(label, sep='|')
    label = tf.strings.to_number(label, out_type=tf.int32)
    label = tf.reduce_sum(tf.one_hot(indices=label, depth=19), axis=0)
    label = tf.reshape(label, (1, 19))
    
    return img, label

In [ ]:


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()



In [ ]:


titles = ['microtubules', 'nuclei', 'endoplasmic reticulum', 'protein of interest']
fig, axs = plt.subplots(3, 4, figsize =(16,8))
for entry in range(3):
    for channel in range(4):
        img = plt.imread(paths[entry][channel])
        axs[entry, channel].imshow(img)        
        if entry == 0:
            axs[0, channel].set_title(titles[channel])



In [ ]:
NUC_MODEL = "./nuclei-model.pth"
CELL_MODEL = "./cell-model.pth"
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    padding=False,
    multi_channel_model=True,
)

image = paths[4]
arrays = image_to_arrays(image)
nuclei = arrays[1]
cell = arrays[:-1]

nuc_segmentations = segmentator.pred_nuclei([nuclei])

inter_step = [[i] for i in image[:-1]]
cell_segmentations = segmentator.pred_cells(inter_step)

In [ ]:


# Nuclei mask
nuclei_mask = label_nuclei(nuc_segmentations[0])
# Cell masks
cell_nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
# Plotting

r = plt.imread(image[0])
b = plt.imread(image[1])
y = plt.imread(image[2])

f, ax = plt.subplots(1, 2, figsize=(16,16))
ax[0].imshow(np.dstack((r,b,y)))
ax[0].set_title('image', size=20)
ax[1].imshow(cell_mask)
ax[1].set_title('Cell Mask', size=20)
plt.show()

